# To do/consider/fix:
- Eventually make code Object oriented?
- Are Jupyter notebooks analyzed in the same way??
- problem in the 'from keras import X, Y, Z', sometimes multiple imports in one line, can't just take last one

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import operator
import numpy as np
%matplotlib inline

auth = open("auth.txt").read()
user, pw = auth.split()[0], auth.split()[1]

## Links
https://stackoverflow.com/questions/10625190/most-suitable-python-library-for-github-api-v3 - PyGithub  
https://stackoverflow.com/questions/42801033/how-to-make-an-url-with-get-repos-owner-repo-labels-name - how to make url from Github GET request
https://stackoverflow.com/questions/43259946/github-api-getting-topics-of-a-github-repository - getting topics  
http://docs.python-requests.org/en/master/user/advanced/ - headers  
http://docs.python-requests.org/en/master/user/authentication/ - Requests Authentication   
https://github.com/nelsonic/github-scraper - go to "Interesting Facts"  
https://developer.github.com/v3/#pagination - need to access multiple pages

https://developer.github.com/v3/repos/#get - github API get request  
https://developer.github.com/v3/search/#search-topics - Github search repos by topic   
https://developer.github.com/v3/guides/traversing-with-pagination/ - Github pagination   
https://stackoverflow.com/questions/33878019/how-to-get-data-from-all-pages-in-github-api-with-python - pagination get next page
https://stackoverflow.com/questions/48390046/download-github-repositories-list-matching-a-topic - search github repos containing a specific topic  

http://docs.python-requests.org/en/v0.10.6/api/ - Requests API

## First, scrape list of github repos with the following topics

In [113]:
topics = ['collision']
max_requests = 4000

n_scraped = 0
for t in topics:
    link = 'https://api.github.com/search/repositories?q=topic:%s&per_page=100'%t
    f = open('repo_data/%s_repos.txt'%t, 'w')
    while link:
        r = requests.get(link, 
                     headers={"Accept":"application/vnd.github.mercy-preview+json"},
                     auth=HTTPBasicAuth(user, pw))
        if(r.ok):
            repos = json.loads(r.text or r.content)
            for item in repos['items']:
                f.write('%s\n'%item['url'])                
                n_scraped += 1 # number of repo urls scraped
        try:
            r_ = r.json()
            link = r.links['next']['url'] # get next page
            print(link)
        except:
            break

        if n_scraped >= max_requests:
            break
    
    f.close()
    print("Scraped %d repos for topic:'%s'"%(n_scraped, t))  # why is it only scraping 4x fewer than I want?

Scraped 86 repos for topic:'collision'


# Features:
- Topics
- Languages
- Popularity info - https://github.com/timqian/star-history - github star history  
- Lines of code - https://codetabs.com/count-loc/count-loc-online.html

In [115]:
topics = ['machine-learning']

for t in topics:
    repo_url_list = open('repo_data/%s_repos.txt'%t, 'r').read().splitlines() 
    data = {}
    for i, repo_url in enumerate(repo_url_list):
        try:
            r = requests.get(repo_url, 
                             headers={"Accept":"application/vnd.github.mercy-preview+json"},
                             auth=HTTPBasicAuth(user, pw))
            repoItem = json.loads(r.text or r.content)
            data[i] = {'topics': repoItem['topics'],
                       'language': repoItem['language'],
                       'stargazers': repoItem['stargazers_count'],
                       'watchers_count': repoItem['watchers_count'],
                       'forks_count': repoItem['forks_count'],
                       'url': repo_url}
        except:
            print('couldnt process github link %s'%repo_url)
    
    df = pd.DataFrame(data).T
    df.to_csv('repo_data/%s_data.csv'%t)

# Scrape actual Code from each repo

In [2]:
df = pd.read_csv('repo_data/machine-learning_data.csv') 

# need to make dtype of list be list
                 #dtype={'forks_count': np.int32, 'stargazers': np.int32, 'watchers_count': np.int32,
                 #      'language': 'str', 'url':'str', 'topics':'list'})
df.head()

,Unnamed: 0,forks_count,language,stargazers,topics,url,watchers_count
0,0,64002,C++,102092,"['deep-learning', 'deep-neural-networks', 'dis...",https://api.github.com/repos/tensorflow/tensor...,102092
1,1,11331,Python,30273,"['data-science', 'deep-learning', 'machine-lea...",https://api.github.com/repos/keras-team/keras,30273
2,2,14206,Python,28627,"['data-analysis', 'data-science', 'machine-lea...",https://api.github.com/repos/scikit-learn/scik...,28627
3,3,14940,C++,24409,"['deep-learning', 'machine-learning', 'vision']",https://api.github.com/repos/BVLC/caffe,24409
4,4,8338,Jupyter Notebook,22789,"['deep-learning', 'examples', 'machine-learnin...",https://api.github.com/repos/aymericdamien/Ten...,22789


In [6]:
df['url'].iloc[2]

'https://api.github.com/repos/scikit-learn/scikit-learn'

In [9]:
repo_url = 'https://api.github.com/repos/scikit-learn/scikit-learn/contents/'
r = requests.get(repo_url,headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(user, pw))
repoItem = json.loads(r.text or r.content)

In [13]:
repoItem

[{'name': '.circleci',
  'path': '.circleci',
  'sha': 'c6b6148f7e14601eafa12025ff4050ad4be740fa',
  'size': 0,
  'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/contents/.circleci?ref=master',
  'html_url': 'https://github.com/scikit-learn/scikit-learn/tree/master/.circleci',
  'git_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/git/trees/c6b6148f7e14601eafa12025ff4050ad4be740fa',
  'download_url': None,
  'type': 'dir',
  '_links': {'self': 'https://api.github.com/repos/scikit-learn/scikit-learn/contents/.circleci?ref=master',
   'git': 'https://api.github.com/repos/scikit-learn/scikit-learn/git/trees/c6b6148f7e14601eafa12025ff4050ad4be740fa',
   'html': 'https://github.com/scikit-learn/scikit-learn/tree/master/.circleci'}},
 {'name': '.codecov.yml',
  'path': '.codecov.yml',
  'sha': '2f73002107a03321b47ed3b982acd9af150fc4a9',
  'size': 672,
  'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/contents/.codecov.yml?ref=master',
  'html_url':

In [178]:
def initialize_metrics(packages = ['numpy', 'keras', 'pandas']):
    d = {}
    for p in packages:
        d[p] = 0
    return d

def update_metrics(item, data):
    text = requests.get(item['download_url'],
                        headers={"Accept":"application/vnd.github.mercy-preview+json"},
                        auth=HTTPBasicAuth(user, pw)).text
    
    for p in data.keys():
        if text.find(p) != -1:    # is package even in script at all?
            # direct import, e.g. 'import numpy (as np)'
            string = 'import %s'%p
            loc = text.find(string)
            if loc != -1:       
                call_name = text[loc:].partition('\n')[0].split()[-1] # last word in line is call_name
                data[p] += text.count('%s.'%call_name)

            # indirect import, e.g. 'from numpy.X import vectorize (as vec)'
            string_from = 'from %s'%p
            loc_from = text.find(string_from)
            if loc_from != -1:  
                while loc_from != -1: # can have multiple imports
                    call_name = text[loc_from:].partition('\n')[0].split()[-1] # problem here, sometimes multiple imports in one line
                    data[p] += sum(text.count('%s%s'%(call_name, x)) for x in '( .'.split()) # check both X() and X.y()
                    loc_from = text.find(string_from, loc_from + 1) # get next instance, e.g. 'from numpy.X import b'
    return data

# recursive, main functions
def digest_repo(repo_url, metrics):
    r = requests.get('%s'%repo_url, headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(user, pw))
    repoItems = json.loads(r.text or r.content)

    for item in repoItems:
        #if item['name'] == 'model_train.py':  #TEMP!!!!!!!!!!!!!!!!!!!!!!!!!
        if item['type'] == 'file' and item['name'][-3:] == '.py':
            metrics = update_metrics(item, metrics)
        elif item['type'] == 'dir':
            metrics = digest_repo(item['url'], metrics)
        print(metrics, item['name'])
    return metrics

In [179]:
repo_url = 'https://api.github.com/repos/silburt/DeepMoon/contents'
metrics = initialize_metrics()
metrics = digest_repo(repo_url, metrics)

{'numpy': 0, 'keras': 0, 'pandas': 0} .gitignore
{'numpy': 0, 'keras': 0, 'pandas': 0} LICENSE.md
{'numpy': 0, 'keras': 0, 'pandas': 0} README.md
{'numpy': 0, 'keras': 0, 'pandas': 0} HeadCraters.csv
{'numpy': 0, 'keras': 0, 'pandas': 0} LROCCraters.csv
{'numpy': 0, 'keras': 0, 'pandas': 0} catalogues
{'numpy': 0, 'keras': 0, 'pandas': 0} CNN1.png
{'numpy': 0, 'keras': 0, 'pandas': 0} CNN2.png
{'numpy': 0, 'keras': 0, 'pandas': 0} Using Zenodo Data.ipynb
{'numpy': 0, 'keras': 0, 'pandas': 0} docs
{'numpy': 16, 'keras': 1, 'pandas': 0} get_unique_craters.py
{'numpy': 51, 'keras': 1, 'pandas': 5} input_data_gen.py
{'numpy': 93, 'keras': 25, 'pandas': 8} model_train.py
{'numpy': 93, 'keras': 25, 'pandas': 8} requirements.txt
{'numpy': 94, 'keras': 25, 'pandas': 8} run_get_unique_craters.py
{'numpy': 94, 'keras': 25, 'pandas': 8} run_input_data_gen.py
{'numpy': 94, 'keras': 25, 'pandas': 8} run_model_train.py
{'numpy': 94, 'keras': 25, 'pandas': 8} LunarLROLrocKaguya_1180mperpix_downsamp.p

In [161]:
metrics

{'numpy': 1, 'keras': 0, 'pandas': 6}

In [108]:
#repo_url = 'https://raw.githubusercontent.com/scikit-learn/scikit-learn/master/benchmarks/bench_20newsgroups.py'
repo_url = 'https://raw.githubusercontent.com/scikit-learn/scikit-learn/master/setup.py'
r = requests.get(repo_url,headers={"Accept":"application/vnd.github.mercy-preview+json"},
                 auth=HTTPBasicAuth(user, pw))

## Building up for now...

In [133]:
packages = ['numpy', 'distutils']

text = r.text
data = {}
for p in packages:
    if text.find(p) != -1:    # is package even in script at all?
        data[p] = 0
        
        # direct import, e.g. 'import numpy (as np)'
        string = 'import %s'%p
        loc = text.find(string)
        if loc != -1:       
            call_name = text[loc:].partition('\n')[0].split()[-1] # last word in line is call_name
            data[p] += text.count('%s.'%call_name)
        
        # indirect import, e.g. 'from numpy.X import vectorize (as vec)'
        string_from = 'from %s'%p
        loc_from = text.find(string_from)
        if loc_from != -1:  
            data[p] = 0
            while loc_from != -1: # can have multiple imports
                call_name = text[loc_from:].partition('\n')[0].split()[-1]
                data[p] += text.count('%s('%call_name)   # find function call
                loc_from = text.find(string_from, loc_from + 1) # get next instance, e.g. 'from numpy.X import b'
                
print(data)

numpy
distutils
{'numpy': 2, 'distutils': 1}


In [112]:
text.splitlines()

['#! /usr/bin/env python',
 '#',
 '# Copyright (C) 2007-2009 Cournapeau David <cournape@gmail.com>',
 '#               2010 Fabian Pedregosa <fabian.pedregosa@inria.fr>',
 '# License: 3-clause BSD',
 'descr = """A set of python modules for machine learning and data mining"""',
 '',
 'import sys',
 'import os',
 'import shutil',
 'from distutils.command.clean import clean as Clean',
 'from pkg_resources import parse_version',
 'import traceback',
 '',
 'if sys.version_info[0] < 3:',
 '    import __builtin__ as builtins',
 'else:',
 '    import builtins',
 '',
 '# This is a bit (!) hackish: we are setting a global variable so that the main',
 '# sklearn __init__ can detect if it is being loaded by the setup routine, to',
 "# avoid attempting to load components that aren't built yet:",
 '# the numpy distutils extensions that are used by scikit-learn to recursively',
 '# build the compiled extensions in sub-packages is based on the Python import',
 '# machinery.',
 'builtins.__SKLEARN_SETU

In [89]:
r.text.find("import numpy as")

86

In [93]:
r.text[86:].partition('\n')[0].split('import numpy as ')[1]

'np'

In [87]:
r.text.splitlines()

['from __future__ import print_function, division',
 'from time import time',
 'import argparse',
 'import numpy as np',
 '',
 'from sklearn.dummy import DummyClassifier',
 '',
 'from sklearn.datasets import fetch_20newsgroups_vectorized',
 'from sklearn.metrics import accuracy_score',
 'from sklearn.utils.validation import check_array',
 '',
 'from sklearn.ensemble import RandomForestClassifier',
 'from sklearn.ensemble import ExtraTreesClassifier',
 'from sklearn.ensemble import AdaBoostClassifier',
 'from sklearn.linear_model import LogisticRegression',
 'from sklearn.naive_bayes import MultinomialNB',
 '',
 'ESTIMATORS = {',
 '    "dummy": DummyClassifier(),',
 '    "random_forest": RandomForestClassifier(n_estimators=100,',
 '                                            max_features="sqrt",',
 '                                            min_samples_split=10),',
 '    "extra_trees": ExtraTreesClassifier(n_estimators=100,',
 '                                        max_features="sqr

In [83]:
string = 'import numpy'
r.text.find(string, 5097, 5097+len(string)+5)

5097